In [1]:
import io
import matplotlib
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import PIL
import torch
import torchvision
import urllib
import sys
import warnings

from torchvision import transforms

sys.path.append(os.path.join(os.path.dirname(os.path.realpath('__file__')), '..'))
from attacks import eot_attacks
from attacks import utils
from utils import labels_util
from utils import datasets

# Disable annoying UserWarning caused by using nn.Upsample
# in the relighting model.
warnings.filterwarnings("ignore", category=UserWarning) 

%load_ext autoreload
%autoreload 2

In [4]:
config = {}
config['classif_model_name'] = 'resnet_indoor'
config['relight_model_name'] = 'multi_illumination_murmann'
config['relighter_eps'] = 1e-4

checkpoint_path = '../'
checkpoint_path += 'relighters/multi_illumination/checkpoints/relight/epoch_13.pth'

relight_model = utils.load_relighting_model(config['relight_model_name'], checkpoint_path)
classif_model = utils.load_classification_model(config['classif_model_name'])

idx_to_label = labels_util.load_idx_to_label('indoor_scenes')
label_to_idx = {label : idx for idx, label in idx_to_label.items()}

print(label_to_idx)

CHECKPOINT FMMMMMMMM:  ../relighters/multi_illumination/checkpoints/relight/epoch_13.pth
Loaded the relighter: multi_illumination_murmann.
{'warehouse': 9, 'inside_subway': 4, 'livingroom': 6, 'restaurant': 7, 'bedroom': 2, 'casino': 3, 'bar': 1, 'kitchen': 5, 'airport_inside': 0, 'subway': 8}


In [5]:
config['target_label'] = label_to_idx['warehouse']  # label for 'warehouse' 
config['debugging'] = False
config['num_iterations'] = 5
config['learning_rate'] = 0.05
config['attack_type'] = 'class_constrained_eot'
config['gamma'] = 1.3

In [10]:
dataset = datasets.IndoorScenesDataset('../data/indoor-scenes/Test.csv', 
                                       '../data/indoor-scenes/')

for i in range(2):
    for idx in range(len(dataset)):
        img, gt_label = dataset[idx]
    
    print('OK!')

OK!
OK!


In [4]:
# correct = 0
# adversarial = 0
# result_has_nan = 0

# dataset = datasets.IndoorScenesDataset('../data/indoor-scenes/Test.csv', 
#                                        '../data/indoor-scenes/')
    
# for eps in [0.01]:
#     config['eps'] = eps
#     correct = 0
#     adversarial = 0
#     result_has_nan = 0

#     save_to_file = True

#     count_classes = {}
#     model_accuracies = []
#     attack_succ_rates = []

#     for idx in range(len(dataset)):
#         img, gt_label = dataset[idx]

#         # Keep the statistics
#         if gt_label in count_classes:
#             count_classes[gt_label] += 1
#         else:
#             count_classes[gt_label] = 1

#         config['gt_label'] = gt_label
#         if gt_label == config['target_label']:
#             config['target_label'] = label_to_idx['kitchen']

#         result = eot_attacks.do_attack(relight_model, classif_model, img, config)

#         if result['orig_label'] == gt_label:
#             correct += 1

#             if result['adv_label'] == config['target_label']:
#                 adversarial += 1

#             # Randomly visualize 25% of the attacks.
#             if np.random.random() < 1. and result['adv_label'] is not None:
#                 eps_str = 'eps_{}_'.format(config['eps'])
#                 utils.visualize_attack(img, result, idx_to_label)

#             print('eps {}, Current index: {}, Correct: {}, Adversarial: {}'.format(
#                 config['eps'], idx, correct, adversarial))

#         accuracy = correct / (idx + 1)
#         attack_succ_rate = adversarial / correct

#         if idx > 0 and \
#            np.abs(accuracy - model_accuracies[-1]) < 1e-4 and \
#            np.abs(attack_succ_rate - attack_succ_rates[-1]) < 1e-4:
#             converged = True

#         model_accuracies.append(accuracy)
#         attack_succ_rates.append(attack_succ_rate)

#         plt.plot(np.arange(len(model_accuracies)), model_accuracies, c='red')
#         plt.title('Classifier accuracy')
#         plt.xlabel('Dataset index')
#         plt.ylabel('Accuracy')
#         plt.close()

#         plt.plot(np.arange(len(attack_succ_rates)), attack_succ_rates, c='blue')
#         plt.title('Attack success rate')
#         plt.xlabel('Dataset index')
#         plt.ylabel('Success rate')
#         plt.close()

In [ ]:
config['attack_type'] = 'random_root'
config['batch_size'] = 32
config['num_batches'] = 50
config['num_classes'] = 10
config['eps'] = 0.02

dataset = datasets.IndoorScenesDataset('../data/indoor-scenes/Test.csv', 
                                       '../data/indoor-scenes/')

for idx in range(len(dataset)):
    img, gt_label = dataset[idx]
    
    config['gt_label'] = gt_label
    if gt_label == config['target_label']:
        config['target_label'] = label_to_idx['bedroom']

    result = eot_attacks.do_attack(relight_model, classif_model, img, config)
    utils.visualize_attack(img, result, idx_to_label)
    
    config['target_label'] = label_to_idx['warehouse']